In [ ]:
# # Dataset: https://www.kaggle.com/datasets/balraj98/apple2orange-dataset
# ! gdown 1eS__rA2XsX0pl_PSnZdunOouEyxYmUar
# !unzip -qq fruit_images.zip

In [ ]:
# On linux, mixed-precision works properly but on windows there's an issue.
# For windows, everything tensorflow related needs to be in the same script or notebook.

In [ ]:
!nvidia-smi

Sat Oct  1 06:36:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !nvidia-smi --query-gpu=utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv

In [ ]:
# !pip install -U watermark livelossplot

In [ ]:
%reload_ext watermark
%watermark -a "Vaibhav Singh" 
%watermark

Author: Vaibhav Singh

Last updated: 2022-10-01T06:36:42.781088+00:00

Python implementation: CPython
Python version       : 3.7.14
IPython version      : 7.9.0

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.10.133+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
# from livelossplot import PlotLossesKerasTF


import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory

from tensorflow.keras.layers import (
    InputLayer,
    Rescaling,
    RandomRotation,
    RandomFlip,
    RandomZoom,
    RandomTranslation,
)

from tensorflow.keras import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

%watermark -iv

numpy     : 1.21.6
tensorflow: 2.8.2+zzzcolab20220719082949
matplotlib: 3.2.2



In [ ]:
# https://www.tensorflow.org/guide/mixed_precision

tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
# from mobilenet_v1 import create_mobilenet_v1
# from mobilenet_v2 import create_mobilenet_v2
from mobilenet_v3 import create_mobilenet_v3

In [ ]:
def set_seeds():
    # fix random seeds
    SEED_VALUE = 1

    np.random.seed(SEED_VALUE)
    tf.keras.utils.set_random_seed(SEED_VALUE)

    # # https://github.com/tensorflow/tensorflow/issues/51978#issuecomment-918917286
    # os.environ["TF_DISABLE_SEGMENT_REDUCTION_OP_DETERMINISM_EXCEPTIONS"] = "True"

    gpus = tf.config.list_physical_devices("GPU")
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            
            # logical_gpus = tf.config.list_logical_devices("GPU")
            # print(logical_gpus)

            # split 1 gpu in 2 logical gpus
            tf.config.set_logical_device_configuration(
                gpus[0], 
                [
                    tf.config.LogicalDeviceConfiguration(memory_limit=2518),
                    tf.config.LogicalDeviceConfiguration(memory_limit=2518),
                    tf.config.LogicalDeviceConfiguration(memory_limit=2518),
                    tf.config.LogicalDeviceConfiguration(memory_limit=2518),
                    tf.config.LogicalDeviceConfiguration(memory_limit=2518),
                    tf.config.LogicalDeviceConfiguration(memory_limit=2518),

                ]
            )

            logical_gpus = tf.config.list_logical_devices("GPU")
            print(logical_gpus)
            print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

        except RuntimeError as e:
            print(e)

    return

set_seeds()

[LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU'), LogicalDevice(name='/device:GPU:2', device_type='GPU'), LogicalDevice(name='/device:GPU:3', device_type='GPU'), LogicalDevice(name='/device:GPU:4', device_type='GPU'), LogicalDevice(name='/device:GPU:5', device_type='GPU')]
1 Physical GPU, 6 Logical GPUs


In [ ]:
@dataclass
class TrainingConfig:
    BATCH_SIZE: int = 128
    EPOCHS: int = 50
    learning_rate: float = 0.001

        
@dataclass
class DatasetConfig:
    NUM_CLASSES: int = 2
    DATA_ROOT: str = "fruit_images"
    SIZE: tuple = (224, 224, 3)

The following code blocks contain all the preprocessing and augmentations that we will apply to the dataset here.

In [ ]:
def data_augmentation_preprocess():

    # Combine multiple augmentations in a single processing pipeline using a list of transformations.
    data_augmentation_pipeline = Sequential(
        [
            RandomRotation(0.25, fill_mode="nearest"),
            RandomFlip("horizontal_and_vertical"),
            RandomZoom(
                height_factor=(-0.25, 0.25),
                width_factor=(-0.25, 0.25),
                fill_mode="nearest",
            ),
            # RandomBrightness((0.5, 0.8)),
            # RandomContrast((0.3, 0.8))
            RandomTranslation(height_factor=(-0.2, 0.3), width_factor=(-0.2, 0.3)),
        ]
    )
    return data_augmentation_pipeline

In [ ]:
def get_data(
    *,
    data_root,
    target_size=(224, 224),
    batch_size=32,
    data_augmentation=True,
):

    train_path = os.path.join(data_root, "Train")
    train_dataset = image_dataset_from_directory(
        train_path,
        label_mode="int",
        color_mode="rgb",
        batch_size=batch_size,
        image_size=target_size,
        shuffle=True,
    )

    valid_path = os.path.join(data_root, "Valid")
    valid_dataset = image_dataset_from_directory(
        valid_path,
        label_mode="int",
        color_mode="rgb",
        batch_size=batch_size,
        image_size=target_size,
        shuffle=False,
    )

    if data_augmentation:
        data_augmentation_pipeline = data_augmentation_preprocess()
        train_dataset = train_dataset.map(
            lambda x, y: (data_augmentation_pipeline(x), y)
        )

    train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
    valid_dataset = valid_dataset.prefetch(tf.data.AUTOTUNE)

    return train_dataset, valid_dataset

In [ ]:
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)

with strategy.scope():
    model = Sequential()
    model.add(InputLayer(input_shape=DatasetConfig.SIZE))
    model.add(Rescaling(1.0 / 255.0))
    model.add(create_mobilenet_v3(
        input_shape=DatasetConfig.SIZE,
        alpha=0.3,
        num_classes=DatasetConfig.NUM_CLASSES,
        pooling="average",
        dropout_rate=0.3,
        large=False,
    ))


    optimizer = Adam(learning_rate=TrainingConfig.learning_rate)

    model.compile(
        optimizer=optimizer,
        loss=SparseCategoricalCrossentropy(from_logits=False),
        metrics=["accuracy"],
    )

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 MobileNet-V3 (Functional)   (None, 2)                 331730    
                                                                 
Total params: 331,730
Trainable params: 327,394
Non-trainable params: 4,336
_________________________________________________________________


In [ ]:
train_dataset, valid_dataset = get_data(
    data_root=DatasetConfig.DATA_ROOT,
    target_size=DatasetConfig.SIZE[:-1],
    batch_size=TrainingConfig.BATCH_SIZE,
    data_augmentation=True
)


history = model.fit(
    train_dataset,
    epochs=TrainingConfig.EPOCHS,
    validation_data=valid_dataset,
    #     callbacks=[
    #         PlotLossesKerasTF(),
    #     ],
    verbose=1,
)

Found 2014 files belonging to 2 classes.
Found 514 files belonging to 2 classes.
Epoch 1/50
16/16 [==============================] - 99s 926ms/step - loss: 0.5550 - accuracy: 0.7304 - val_loss: 0.6938 - val_accuracy: 0.4825
Epoch 2/50
16/16 [==============================] - 7s 389ms/step - loss: 0.4359 - accuracy: 0.8734 - val_loss: 0.6950 - val_accuracy: 0.4825
Epoch 3/50
16/16 [==============================] - 11s 609ms/step - loss: 0.4145 - accuracy: 0.8947 - val_loss: 0.6959 - val_accuracy: 0.4825
Epoch 4/50
16/16 [==============================] - 7s 391ms/step - loss: 0.4151 - accuracy: 0.8937 - val_loss: 0.6968 - val_accuracy: 0.4825
Epoch 5/50
16/16 [==============================] - 7s 382ms/step - loss: 0.4113 - accuracy: 0.8952 - val_loss: 0.6989 - val_accuracy: 0.4825
Epoch 6/50
16/16 [==============================] - 7s 388ms/step - loss: 0.4018 - accuracy: 0.9081 - val_loss: 0.6977 - val_accuracy: 0.4825
Epoch 7/50
16/16 [==============================] - 7s 385ms/step